# Gate infidelity calculation
This notebook uses physical value to calculate the gate infidelity of a two-qubit spin gate for electrons in Penning trap. The motional mode is assumed to be the axial mode.

Parameters:
Magnetic field gradient 250 T/m at 50 $\mu \mathrm{m}$ above the surface.
$\frac{d^3B}{dz^3}=-7.2\times 10^9\ \mathrm{T/m}^3$.

We scan the axial frequency from 10 MHz to 1GHz.